In [1]:
import torch
import matplotlib.pyplot as plt
import sys

In [2]:
from xno.models import XNO
from xno.data.datasets import Burgers1dTimeDataset
from xno.utils import count_model_params
from xno.training import AdamW
from xno.training.incremental import IncrementalFNOTrainer
from xno.data.transforms.data_processors import IncrementalDataProcessor
from xno import LpLoss, H1Loss
from torch.utils.data import DataLoader
from pathlib import Path

In [3]:
from torch.utils.data import DataLoader, TensorDataset, Dataset, default_collate
from utils import MatReader

# Loading Burgers 1D dataset

In [4]:
ntrain = 1000
ntest = 100

sub = 2**3 #subsampling rate
h = 2**13 // sub #total grid size divided by the subsampling rate
s = h

batch_size = 20
learning_rate = 0.001

epochs = 500
step_size = 50
gamma = 0.5

modes = 16
width = 64

In [5]:
dataloader = MatReader('data/burgers_data_R10.mat')
x_data = dataloader.read_field('a')[:,::sub]
y_data = dataloader.read_field('u')[:,::sub]

x_train = x_data[:ntrain,:]
y_train = y_data[:ntrain,:]
x_test = x_data[-ntest:,:]
y_test = y_data[-ntest:,:]

x_train = x_train.reshape(ntrain,s,1)
x_test = x_test.reshape(ntest,s,1)

train_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_train, y_train), batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(torch.utils.data.TensorDataset(x_test, y_test), batch_size=batch_size, shuffle=False)

In [6]:
# Define the custom Dataset
class DictDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return {'x': self.x[idx], 'y': self.y[idx]}

In [ ]:
x_train.shape, y_train.shape

In [ ]:
x_test.shape, y_test.shape

In [9]:
x_train = x_train.permute(0, 2, 1)
y_train = y_train.unsqueeze(1)
x_vali = x_test.permute(0, 2, 1)
y_vali = y_test.unsqueeze(1)

In [10]:
train_loader = DictDataset(x_train, y_train)
test_loader = DictDataset(x_vali, y_vali)

In [11]:
train_loader = DataLoader(train_loader, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_loader, batch_size=batch_size, shuffle=True)
test_loader = {
    1024: test_loader
}

In [ ]:
batch = next(iter(train_loader))
type(train_loader), type(batch), batch['x'].shape, batch['y'].shape

In [ ]:
batch = next(iter(test_loader[1024]))
type(test_loader[1024]), type(batch), batch['x'].shape, batch['y'].shape

In [14]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
model = XNO(
    max_n_modes=(16, ),
    n_modes=(modes, ),
    hidden_channels=width,
    in_channels=1,
    out_channels=1,
    transformation="wno",
    transformation_kwargs={"wavelet_level": 3, "wavelet_size": [16]}
)
model = model.to(device)
n_params = count_model_params(model)

In [16]:
optimizer = AdamW(model.parameters(), lr=8e-3, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)

In [ ]:
data_transform = IncrementalDataProcessor(
    in_normalizer=None,
    out_normalizer=None,
    device=device,
    subsampling_rates=[2, 1],
    dataset_resolution=16,
    dataset_indices=[2],
    epoch_gap=10,
    verbose=True,
)

data_transform = data_transform.to(device)

In [ ]:
l2loss = LpLoss(d=2, p=2)
h1loss = H1Loss(d=2)
train_loss = h1loss
eval_losses = {"h1": h1loss, "l2": l2loss}
print("\n### N PARAMS ###\n", n_params)
print("\n### OPTIMIZER ###\n", optimizer)
print("\n### SCHEDULER ###\n", scheduler)
print("\n### LOSSES ###")
print("\n### INCREMENTAL RESOLUTION + GRADIENT EXPLAINED ###")
print(f"\n * Train: {train_loss}")
print(f"\n * Test: {eval_losses}")
sys.stdout.flush()

In [19]:
# Finally pass all of these to the Trainer
trainer = IncrementalFNOTrainer(
    model=model,
    n_epochs=10,
    data_processor=data_transform,
    device=device,
    verbose=True,
    incremental_loss_gap=False,
    incremental_grad=True,
    incremental_grad_eps=0.9999,
    incremental_loss_eps = 0.001,
    incremental_buffer=5,
    incremental_max_iter=1,
    incremental_grad_max_iter=2,
)

In [ ]:
trainer.train(
    train_loader,
    test_loader,
    optimizer,
    scheduler,
    regularizer=False,
    training_loss=train_loss,
    eval_losses=eval_losses,
)

In [ ]:
# FNO
{'train_err': 0.9652893996238708,
 'avg_loss': 0.04826446998119354,
 'avg_lasso_loss': None,
 'epoch_train_time': 27.041632332999143,
 '1024_h1': tensor(0.0500),
 '1024_l2': tensor(0.0418)}

In [22]:
# HNO
{'train_err': 1.0265669953823089,
 'avg_loss': 0.05132834976911545,
 'avg_lasso_loss': None,
 'epoch_train_time': 34.57337279099738,
 '1024_h1': tensor(0.0500),
 '1024_l2': tensor(0.0445)}

In [23]:
# LNO


In [24]:
# WNO